In [1]:
import osmnx as ox
import csv

In [2]:
# Center of Jyväskylä
center_point = (62.24147, 25.72088)
dist = 5000

# Rich road types
road_types = ['motorway', 'trunk', 'primary', 'secondary', 'tertiary',
              'residential', 'unclassified', 'living_street']

custom_filter = '["highway"~"{}"]'.format('|'.join(road_types))

# Load graph
G = ox.graph_from_point(center_point, dist=dist, network_type='drive')

In [3]:
self_loops = [(u, v) for u, v, data in G.edges(data=True) if u == v]
G.remove_edges_from(self_loops)

In [4]:
# Normalize coordinates
# Get the min and max coordinates for scaling
x_coords = [data['x'] for _, data in G.nodes(data=True)]
y_coords = [data['y'] for _, data in G.nodes(data=True)]

x_min, x_max = min(x_coords), max(x_coords)
y_min, y_max = min(y_coords), max(y_coords)

# Define window size for normalization (e.g., 800x600)
window_width = 1280
window_height = 720

# Normalize coordinates
for node_id, data in G.nodes(data=True):
    normalized_x = (data['x'] - x_min) / (x_max - x_min) * window_width
    normalized_y = (data['y'] - y_min) / (y_max - y_min) * window_height
    data['x'] = normalized_x
    data['y'] = normalized_y

In [5]:
id_map = {orig_id: i for i, orig_id in enumerate(G.nodes())}

In [6]:
with open("nodes.csv", "w", newline='') as f_nodes:
    writer = csv.writer(f_nodes)
    for orig_id, data in G.nodes(data=True):
        writer.writerow([id_map[orig_id], data["x"], data["y"]])

In [8]:
with open("edges.csv", "w", newline='', encoding='utf-8') as f_edges:
    writer = csv.writer(f_edges)
    for u, v, data in G.edges(data=True):
        name = data.get("name", "Unnamed Road")
        if isinstance(name, list):
            name = name[0]
        writer.writerow([
            id_map[u],
            id_map[v],
            data.get("length", 0.0),
            name
        ])